In [1]:
import pandas as pd

In [2]:
question = pd.read_csv('Janwaar_Questionnaire_22b24a43-67f1-4360-b99d-eb8d750b7772/Janwaar_Questionnaire.csv')

In [3]:
resp1 = pd.read_csv('Janwaar_Questionnaire_22b24a43-67f1-4360-b99d-eb8d750b7772/परिवार_के_सदस्य_के_बारे_में_विवरण_3Bz5ahI2iJ.csv')

In [4]:
resp2 = pd.read_csv('Janwaar_Questionnaire_22b24a43-67f1-4360-b99d-eb8d750b7772/राशन_के_बारे_में_विवरण_1D0ilwHEDE.csv')

In [5]:
x = pd.ExcelFile('Eligibility criteria - worksheet.xlsx')

In [6]:
# The eligibility criteria
df1 = x.parse('Sheet1')

In [7]:
# to make handling nan/na values easy assign a random negative integer to NA entries
df1 = df1.fillna(-8238124)
# get a list of actionable schemes with their index and criteria
action = []
for i, scheme in df1.iterrows():
    temp = []
    for key, val in scheme[9:].items():
        if val != -8238124:
            temp.append((key,val))
    action.append((i, temp))

In [29]:
resp1.groupby('member_gender').count()

,Response ID,Group Context,member_name,member_age,member_relation_hoh,member_marital_status,member_highest_education,member_studying_yn,member_aadhaar_card_yn,member_aadhaar_card_number,...,member_major_illness_इतर,member_major_illness_नहीं,major_illness_other,member_intoxication_yn_नहीं,member_intoxication_yn_हाँ - तम्बाकू,member_intoxication_yn_हाँ - दारु,member_intoxication_yn_हाँ - महुआ,member_intoxication_yn_हाँ - गांजा,2xEmY7oaKy_इनका व्यवसाय के बारे में थोडा स्कप्ष्ट कीजिये,"2xEmY7oaKy_अगर हाँ, तोह किस तरह की विकलांगता?"
member_gender,,,,,,,,,,,,,,,,,,,,,
पुरुष,249,249,249,249,249,196,249,202,249,207,...,249,249,8,149,149,149,149,149,0,0
स्त्री,207,207,207,207,207,169,207,177,207,180,...,207,207,9,131,131,131,131,131,0,0


In [8]:
matches = []
g_count = {}
for i, criteria in action:
#     print(i, criteria)
    if True:
        g_count[i] = 0
        for _, name in resp1.iterrows():
            # temp holds the current criteria, if all the requirements matches, i.e temp goes to 0 then its a match.
            temp = len(criteria)
            for c in criteria:
                if c[0] == 'ration_card':
                    # finding family ration card
                    try:
                        r_c = (question[question['Response ID'] == name['Response ID']].ration_card.values[0])
                        if c[1] == r_c:
                            temp -= 1
                    except:
                        print('ration card not found for ', name.member_name,name['Response ID'])
                    

                elif c[0] == 'member_age':
                    # Age parser and matching scheme
                    if (c[1].find('>')) != -1:
                        pointer = c[1].find('>')
                        if name.member_age >= int(c[1][pointer+1:]):
                            temp -= 1
                    elif (c[1].find('<')) != -1:
                        pointer = c[1].find('<')
                        if name.member_age <= int(c[1][pointer+1:]):
                            temp -= 1
                    elif (c[1].find('-')) != -1:
                        pointer = c[1].find('-')
                        if int(c[1][:pointer]) <= name.member_age and int(c[1][pointer+1:]) >= name.member_age:
                            temp -= 1
                    else:
                        # Dom't have a case yet
                        pass

                elif c[0] == 'member_gender':
                    if c[1] == 'स्त्री' and name.member_gender == 'स्त्री':
                        temp -= 1
                    elif c[1] == 'पुरुष' and name.member_gender == 'पुरुष':
                        temp -= 1
                    else:
                        # Don't have a case yet
                        pass

                elif c[0] == 'member_disabled_yn':
                    if c[1].lower() == 'yes' and name.member_disabled_yn.lower() == 'yes':
                        temp -= 1

                elif c[0] == 'member_disability_percentage':
                    # check this again
                    if c[1] == name.member_disability_percentage:
                        temp -= 1

                elif c[0] == 'member_marital_status':
                    # check this again
                    # multiple possiblites 
                    status = c[1].split('|')
                    for s in status:
                        if s == name.member_marital_status:
                            temp -= 1
                            break

                elif c[0] == 'member_disability_type':
                    if c[1] == 'बौद्धिक' and name['member_disability_type_बौद्धिक'] == 1.:
                        temp -= 1
                    else:
                        # no case yet
                        pass

                elif c[0] == 'toilet_yn':
                    bool_toilet = (question[question['Response ID'] == name['Response ID']].toilet_yn.values[0])
                    if c[1] == 'नहीं' and bool_toilet == 'नहीं':
                        temp -= 1

                elif c[0] == 'caste':
                    # get caste 
                    caste = (question[question['Response ID'] == name['Response ID']].caste.values[0])
                    bpl = (question[question['Response ID'] == name['Response ID']].ration_card.values[0])
                    if c[1] == 'SC - अनुसूचित जाती/ST - अनुसूचित जनजाति (If SC/ST, need not be BPL)':
                        # check this again
                        if caste == 'ST - अनुसूचित जनजाति' or caste == 'SC - अनुसूचित जाती':
                            if bpl == 'bpl':
                                temp -= 1
                            # to make it pass the need not be BPL case
                            elif 'ration_card' in criteria:
                                temp -= 2
                        else:
                            temp -= 1
                    elif c[1] == 'ST - अनुसूचित जनजाति' and caste == 'ST - अनुसूचित जनजाति':
                        temp -= 1
                    elif c[1] == 'SC - अनुसूचित जाती' and caste == 'SC - अनुसूचित जाती':
                        # apparently no SC in the db
                        temp -=1
                    else:
                        # no case yet
                        pass

                # this thing may require manual stuff 
                elif c[0] == 'member_major_illness':
                    list_of_illness_covered = set(c[1].split('|'))
                    list_of_illness_member_has = set()
                    if name.member_major_illness_नहीं == 0:
                        if name.member_major_illness_हृदयरोग == 1:
                            list_of_illness_member_has.add('हृदयरोग')
                        if name.member_major_illness_किडनी == 1:
                            list_of_illness_member_has.add('किडनी')
                        if name.member_major_illness_कैंसर == 1:
                            list_of_illness_member_has.add('कैंसर')
                        if name.member_major_illness_इतर == 1:
                            list_of_illness_member_has.add('इतर')
                    intersec = list_of_illness_covered.intersection(list_of_illness_member_has)
                    if len(intersec) > 0:
                        temp -= 1

                elif c[0] == 'member_pregnant_yn':
                    if c[1] == 'हाँ' and name.member_pregnant_yn == 'हाँ':
                        temp -= 1

                elif c[0] == 'member_highest_education':
                    qual = set(str(c[1]).split('|'))
                    if name.member_highest_education in qual:
                        temp -= 1

                elif c[0] == 'member_studying_yn':
                    if name.member_studying_yn == 'Yes' and c[1] == 'Yes':
                        temp -= 1
    
                    elif c[1] != 'Yes':
                        # no case yet
                        pass
                    else:
                        # no case yet
                        pass
                    

                elif c[0] == 'household_primary_source_income':
                    income = set(c[1].split('|'))
                    list_of_income = set()
                    household = question[question['Response ID'] == name['Response ID']]
                    if household.household_primary_source_income_इतर.values[0] == 1:
                        list_of_income.add('इतर')
                    if household.household_primary_source_income_कृषि.values[0] == 1:
                        list_of_income.add('कृषि')
                    if household.household_primary_source_income_गवर्नमेंट.values[0] == 1:
                        list_of_income.add('गवर्नमेंट')
                    if household.household_primary_source_income_पशुपालन.values[0] == 1:
                        list_of_income.add('पशुपालन')
                    if household.household_primary_source_income_लेबर.values[0] == 1:
                        list_of_income.add('लेबर')
                    if household['household_primary_source_income_तेंदू पत्ता collector'].values[0] == 1:
                        list_of_income.add('तेंदू पत्ता collector')
                    if household['household_primary_source_income_मैन्युअल सफाई'].values[0] == 1:
                        list_of_income.add('मैन्युअल सफाई')
                    if household['household_primary_source_income_कंस्ट्रक्शन लेबर'].values[0] == 1:
                        list_of_income.add('कंस्ट्रक्शन लेबर')
                    if household['household_primary_source_income_गवर्नमेंट - temporary'].values[0] == 1:
                        list_of_income.add('गवर्नमेंट - temporary')
                    if household['household_primary_source_income_wood collector'].values[0] == 1:
                        list_of_income.add('wood collector')
                    if household['household_primary_source_income_चर्मशोधक'].values[0] == 1:
                        list_of_income.add('चर्मशोधक')
                        
                    if len(income.intersection(list_of_income)) > 0:
                        temp -= 1

                elif c[0] == 'member_occupation':
                    occupation = set(c[1].split('|'))
                    if name.member_occupation in occupation:
                        temp -= 1

                elif c[0] == 'household_income_monthly':
                    income = question[question['Response ID'] == name['Response ID']].household_income_monthly.values[0]
                    if int(income) <= int(c[1][1:]):
                        temp -= 1

                elif c[0] == 'member_relation_hoh':
                    if c[1] == 'Self' and name.member_relation_hoh == 'Self':
                        temp -= 1
                    else:
                        # no case yet
                        pass

                elif c[0] == 'member_aadhaar_card_yn':
                    if c[1] == 'No' and name.member_aadhaar_card_yn == 'No':
                        temp -= 1
                    elif c[1] == 'Yes' and name.member_aadhaar_card_yn == 'Yes':
                        temp -= 1
                    else:
                        pass
                
                elif c[0] == 'religion':
                    rel = question[question['Response ID'] == name['Response ID']].religion.values[0]
                    req_rel = set(c[1].split('|'))
                    if rel in req_rel or rel == 'Adivasi':
                        temp -= 1
                
                elif c[0] == 'farming_land':
                    land = set(c[1].split('|'))
                    land_use = set()
                    farm_land = question[question['Response ID'] == name['Response ID']]
                    if farm_land['farming_land_अपनी खेती खुद के ज़मीन पर'].values[0] == 1.0:
                        land_use.add('अपनी खेती खुद के ज़मीन पर')
                    if farm_land['farming_land_अपनी खेती किराया ज़मीन पर'].values[0] == 1.0:
                        land_use.add('अपनी खेती किराया ज़मीन पर')
                    if farm_land['farming_land_किसी और के लिए'].values[0] == 1.0:
                        land_use.add('किसी और के लिए')
                    if len(land.intersection(land_use)) > 0:
                        temp -= 1                    
                
                elif c[0] == 'farming_crops':
                    farm = question[question['Response ID'] == name['Response ID']]
                    farm_set = set()
                    req_farm = set(c[1].split('|'))
                    if (farm['farming_crops_oilseed (तिलहन)'].values[0]) == 1:
                        farm_set.add('oilseed (तिलहन)')
                    if (farm['farming_crops_pulses (दलहन)'].values[0]) == 1:
                        farm_set.add('pulses (दलहन)')
                    if (farm['farming_crops_groundnut (मूंगफली)'].values[0]) == 1:
                        farm_set.add('groundnut (मूंगफली)')
                    if (farm['farming_crops_mustard (सरसों)'].values[0]) == 1:
                        farm_set.add('mustard (सरसों)')
                    if (farm['farming_crops_sugarcane (गन्ना)'].values[0]) == 1:
                        farm_set.add('sugarcane (गन्ना)')
                    if (farm['farming_crops_maize (मक्का)'].values[0]) == 1:
                        farm_set.add('maize (मक्का)')
                    if (farm['farming_crops_rice (चावल)'].values[0]) == 1:
                        farm_set.add('rice (चावल)')
                    if (farm['farming_crops_wheat (गेहू)'].values[0]) == 1:
                        farm_set.add('wheat (गेहू)')
                    if (farm['farming_crops_इतर'].values[0]) == 1:
                        farm_set.add('इतर')
                    if (farm['farming_crops_sunflower (सूरजमुखी)'].values[0]) == 1:
                        farm_set.add('sunflower (सूरजमुखी)')
                    if (farm['farming_crops_लौकी'].values[0]) == 1:
                        farm_set.add('लौकी')
                    if (farm['farming_crops_ककड़ी'].values[0]) == 1:
                        farm_set.add('ककड़ी')
                    if (farm['farming_crops_अमरुद'].values[0]) == 1:
                        farm_set.add('अमरुद')
                    if (farm['farming_crops_sunflower (सुन्फ्लोवेर)'].values[0]) == 1:
                        farm_set.add('sunflower (सुन्फ्लोवेर)')
                    farm_set.add(farm['farming_crops_others'].values[0])
                    if (farm['farming_crops_soyabean (सोयाबीन)'].values[0]) == 1:
                        farm_set.add('soyabean (सोयाबीन)')
                    
                    if len(farm_set.intersection(req_farm)) > 0:
                        temp -=1
                    
                # for some random reason there needs to be a space here after the condition    
                elif c[0] == 'livestock_animals ':
                    house_animal = question[question['Response ID'] == name['Response ID']]
                    req_animals = set(c[1].split('|'))
                    animals = set()
                    if house_animal['livestock_animals_गाय'].values[0] == 1:
                        animals.add('गाय')
                    if house_animal['livestock_animals_बैल'].values[0] == 1:
                        animals.add('बैल')
                    if house_animal['livestock_animals_भैंस'].values[0] == 1:
                        animals.add('भैंस')
                    if house_animal['livestock_animals_मुर्गी'].values[0] == 1:
                        animals.add('मुर्गी')
                    if house_animal['livestock_animals_बकरी/भेड'].values[0] == 1:
                        animals.add('बकरी/भेड')
                    if house_animal['livestock_animals_सूअर'].values[0] == 1:
                        animals.add('सूअर')
                    if house_animal['livestock_animals_इतर'].values[0] == 1:
                        animals.add('इतर')
                    if len(req_animals.intersection(animals)) > 0:
                        temp -= 1
                
                elif c[0] == 'member_nrega_number_days':
                    # for scheme 26
                    if c[1] == '>15 (This OR the conditions in member_source_of_income) ':
                        # using the occupation we define earlier and to take care of the OR condition
                        if name.member_nrega_number_days > 15 and name.member_occupation not in occupation:
                            temp -= 2
                        elif name.member_nrega_number_days > 15:
                            temp -= 1                        

            # check if all criteria matches, if temp goes to zero then voila it's a match
            if temp == 0:
                matches.append((name.member_name, df1.iloc[i], i+1, name))
                g_count[i] += 1
            
            if temp == 1:
                almsot_matches.append((name.member_name, df1.iloc[i], i+1, name))

In [11]:
matches[0]

('Mahesh gond',
 S.No.                                                                              1
 Done by                                                                      Avinash
 Masterlist no.                                                                   303
 Scheme Name                                       IG National Old Age Pension Scheme
 Scheme benefits                    300 rupees per month if age between 60 and 80....
 Scheme (State/Central)                                                 Central+State
 Information link                   http://pensions.samagra.gov.in/IGNOAPDetails.aspx
 Scheme Name.1                                     IG National Old Age Pension Scheme
 Scheme type - HH or individual                                            Individual
 member_age                                                                       >60
 ration_card                                                                      BPL
 member_disabled_yn                   

len(matches)

In [122]:
# 11, 16

In [123]:
g_count

{0: 22,
 1: 1,
 2: 5,
 3: 22,
 4: 8,
 5: 1,
 6: 1,
 7: 3,
 8: 63,
 9: 287,
 10: 1,
 11: 0,
 12: 105,
 13: 69,
 14: 2,
 15: 3,
 16: 3,
 17: 1,
 18: 2,
 19: 54,
 20: 0,
 21: 0,
 22: 4,
 23: 63,
 24: 217,
 25: 14,
 26: 51,
 27: 320,
 28: 69,
 29: 20,
 30: 4,
 31: 0,
 32: 95,
 33: 146,
 34: 0,
 35: 351,
 36: 166,
 37: 456,
 38: 167,
 39: 116,
 40: 15,
 41: 0,
 42: 14,
 43: 116,
 44: 66,
 45: 94,
 46: 17,
 47: 116,
 48: 116,
 49: 104,
 50: 104,
 51: 82,
 52: 0,
 53: 0,
 54: 0,
 55: 2,
 56: 1,
 57: 8,
 58: 13,
 59: 11,
 60: 3,
 61: 1,
 62: 105,
 63: 1,
 64: 0}

In [73]:
df1.iloc[11]

S.No.                                                                             12
Done by                                                                      Avinash
Masterlist no.                                                                    15
Scheme Name                                    Mukhyamantri Bal Hriday Upchar Yojana
Scheme benefits                    Chief Minister Child Heart Treatment Scheme be...
Scheme (State/Central)                                                         State
Information link                   https://govinfo.me/chief-minister-child-heart-...
Scheme Name.1                                  Mukhyamantri Bal Hriday Upchar Yojana
Scheme type - HH or individual                                            Individual
member_age                                                                      0-15
ration_card                                                                 -8238124
member_disabled_yn                                               

In [75]:
resp1[(resp1.member_major_illness_हृदयरोग == 1)].member_age

91     49
311    20
417    84
418    74
Name: member_age, dtype: int64

In [124]:
matches[0][1]

S.No.                                                                              1
Done by                                                                      Avinash
Masterlist no.                                                                   303
Scheme Name                                       IG National Old Age Pension Scheme
Scheme benefits                    300 rupees per month if age between 60 and 80....
Scheme (State/Central)                                                 Central+State
Information link                   http://pensions.samagra.gov.in/IGNOAPDetails.aspx
Scheme Name.1                                     IG National Old Age Pension Scheme
Scheme type - HH or individual                                            Individual
member_age                                                                       >60
ration_card                                                                      BPL
member_disabled_yn                                               

In [141]:
family_match = {}
for i, j, k, l in matches:
    f_id = l['Response ID']
    
    if f_id not in family_match:
        family_match[f_id] = {}
        family_match[f_id]['household'] = set()
        if j['Scheme type - HH or individual'] == 'Individual':
            family_match[f_id][i] = [(j['Scheme Name.1'], k)]
        else:
            family_match[f_id]['household'].add((j['Scheme Name.1'], k))
    else:
        if j['Scheme type - HH or individual'] == 'Individual':
            if i not in family_match[f_id]:
                family_match[f_id][i] = [(j['Scheme Name.1'], k)]
            else:
                family_match[f_id][i].append((j['Scheme Name.1'], k))
        else:
            family_match[f_id]['household'].add((j['Scheme Name.1'], k))

In [137]:
question.house_number.unique()

array(['M1', 'M3', 'M5', 'M7', 'M8', 'm2', 'M4', 'M6', 'm10', 'm11', 'm18',
       'm15', 'm16', 'm30', 'm32', 'aa1', 'aa2', 'm33', 'm37', 'm36',
       'm31', 'm17', 'm14', 'm13', 'm19', 'm38', 'm60', 'm57', 'm55',
       'm51', 'm49', 'm48', 'm47', 'm45', 'm43', 'm42', 'm41', 'm58',
       'm39', 'm80', 'm79', 'm78', 'm77', 'm73', 'm75', 'm74', 'm71',
       'm70', 'm76', 'm54', 'm53', 'm52', 'AV14', 'AV13', 'AV12', 'AV10',
       'm50', 'AV6', 'AV8', 'AV4', 'AV5', 'AV3', 'AV2', 'm40', 'M16',
       'm20', 'm21', 'm34'], dtype=object)

In [79]:
match_dict = {}
ulta_match = {}
for i, j, k, l in matches:
    if i in match_dict:
        match_dict[i].append(j)
    else:
        match_dict[i] = [j]
    if k in ulta_match:
        ulta_match[k].append(i)
    else:
        ulta_match[k] = [i]

#### Writing to file

In [164]:
for key, val in family_match.items():
    f_name = str(question[question['Response ID'] == key].hoh_name.values[0]) + ' '+str(question[question['Response ID'] == key].house_number.values[0]) 
    with open('individual_family/'+f_name+'.txt' , 'w') as f:
        for i, j in val.items():
            t = 'Member Name: ' + str(i) + '\n' 
            f.write(t)
            for v in j:
                f.write('Scheme Name: ' + str(v[0]) + ', Scheme ID: ' + str(v[1]) + '\n')
            f.write('\n')
            f.write('----------------------------------------------------------')
            f.write('\n')

In [81]:
with open('preresults.txt', 'w') as f:
    for key, val in match_dict.items():
        t = '--' + str(key) + '--' + '\n' 
        f.write(t)
        for v in val:
            f.write(str(v)+ '\n')
        f.write('\n')

In [96]:
len(set(resp1.member_name.values).intersection(set(match_dict.keys())))

446

'Guman Adiwasi M1'

In [67]:
question[question.ration_card == 'AAY (Antyodaya Anna Yojana Card)'].household_income_monthly

65    9000
69    1000
81    2000
Name: household_income_monthly, dtype: int64

In [342]:
question[question['Response ID'] == '041b815c-1b43-45dc-b7cf-60b4deebb8c0'].caste

68    ST - अनुसूचित जनजाति
Name: caste, dtype: object

In [27]:
question[question['Response ID'] == '041b815c-1b43-45dc-b7cf-60b4deebb8c0'].ration_card

68    BPL
Name: ration_card, dtype: object

In [274]:
resp1.member_majo

array(['ST - अनुसूचित जनजाति', 'OBC - अन्य पिछड़ी जाती', 'General', nan], dtype=object)

In [285]:
resp1.member_major_illness_इतर.unique()

array([0, 1])

In [286]:
resp1.member_major_illness_किडनी.unique()

array([0])

In [287]:
resp1.member_major_illness_कैंसर.unique()

array([0, 1])

In [295]:
resp1[resp1.member_major_illness_हृदयरोग == 1]

,Response ID,Group Context,member_name,member_gender,member_age,member_relation_hoh,member_marital_status,member_highest_education,member_studying_yn,member_aadhaar_card_yn,...,member_major_illness_इतर,member_major_illness_नहीं,major_illness_other,member_intoxication_yn_नहीं,member_intoxication_yn_हाँ - तम्बाकू,member_intoxication_yn_हाँ - दारु,member_intoxication_yn_हाँ - महुआ,member_intoxication_yn_हाँ - गांजा,2xEmY7oaKy_इनका व्यवसाय के बारे में थोडा स्कप्ष्ट कीजिये,"2xEmY7oaKy_अगर हाँ, तोह किस तरह की विकलांगता?"
91,c13f258f-5efd-480f-93df-9c595af72a26,2,mithkesh kumari ray,स्त्री,49,Spouse,Married,Illiterate,No,Yes,...,0,0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN
311,041b815c-1b43-45dc-b7cf-60b4deebb8c0,8,Ravi Adiwasi,पुरुष,20,Child,Unmarried,Illiterate,No,Yes,...,0,0,NaN,0.0,1.0,0.0,0.0,0.0,NaN,NaN
417,3211d577-9212-4286-a505-9d39d1fefa57,1,halke yadav,पुरुष,84,Self,Married,Illiterate,No,Yes,...,0,0,NaN,0.0,1.0,0.0,0.0,0.0,NaN,NaN
418,3211d577-9212-4286-a505-9d39d1fefa57,2,pyari bai yadav,स्त्री,74,Spouse,Married,Illiterate,No,Yes,...,0,0,NaN,1.0,1.0,0.0,0.0,0.0,NaN,NaN


In [20]:
resp1[(resp1.member_pregnant_yn=='हाँ') & (resp1.member_gender == 'स्त्री')]

,Response ID,Group Context,member_name,member_gender,member_age,member_relation_hoh,member_marital_status,member_highest_education,member_studying_yn,member_aadhaar_card_yn,...,member_major_illness_इतर,member_major_illness_नहीं,major_illness_other,member_intoxication_yn_नहीं,member_intoxication_yn_हाँ - तम्बाकू,member_intoxication_yn_हाँ - दारु,member_intoxication_yn_हाँ - महुआ,member_intoxication_yn_हाँ - गांजा,2xEmY7oaKy_इनका व्यवसाय के बारे में थोडा स्कप्ष्ट कीजिये,"2xEmY7oaKy_अगर हाँ, तोह किस तरह की विकलांगता?"
110,b33f5176-92b2-495b-83ea-7f6aeeea1d15,3,shakuntala gond,स्त्री,19,Son/daughter-in-law,Married,Illiterate,No,No,...,0,1,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN
150,36353fc3-90b8-425f-8008-15cf2523a0b0,2,rani bai gond,स्त्री,29,Spouse,Married,5,No,Yes,...,0,1,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN
307,041b815c-1b43-45dc-b7cf-60b4deebb8c0,4,Choti Adiwasi,स्त्री,18,Son/daughter-in-law,Married,Illiterate,No,Yes,...,0,1,NaN,0.0,1.0,0.0,0.0,0.0,NaN,NaN


In [324]:
    resp1.member_studying_yn.unique()

array(['No', nan, 'Yes'], dtype=object)

In [370]:
resp1.member_occupation.unique()

array(['इतर', 'नहीं', 'कंस्ट्रक्शन लेबर', 'कृषि', 'घर का काम', nan,
       'पशुपालन', 'wood collector'], dtype=object)

In [408]:
resp1.member_aadhaar_card_yn

0      Yes
1      Yes
2       No
3       No
4       No
5       No
6      Yes
7      Yes
8       No
9       No
10      No
11      No
12     Yes
13     Yes
14     Yes
15     Yes
16     Yes
17     Yes
18     Yes
19     Yes
20     Yes
21     Yes
22     Yes
23     Yes
24     Yes
25     Yes
26     Yes
27     Yes
28      No
29      No
      ... 
426    Yes
427    Yes
428    Yes
429     No
430    Yes
431    Yes
432    Yes
433    Yes
434    Yes
435     No
436     No
437    Yes
438    Yes
439    Yes
440    Yes
441     No
442    Yes
443    Yes
444     No
445    Yes
446    Yes
447    Yes
448    Yes
449    Yes
450    Yes
451    Yes
452    Yes
453    Yes
454    Yes
455    Yes
Name: member_aadhaar_card_yn, dtype: object